In [ ]:
import panel as pn
pn.extension()

One of the main design goals for Panel was that it should make it possible to seamlessly transition back and forth between interactively prototyping a dashboard in the notebook or on the commandline to deploying it as a standalone server app. This section shows how to display panels interactively, embed static output, save a snapshot, and deploy as a separate web-server app.

## Configuring output

As you may have noticed, almost all the Panel documentation is written using notebooks. Panel objects display themselves automatically in a notebook and take advantage of Jupyter Comms to support communication between the rendered app and the Jupyter kernel that backs it on the Python end. To display a Panel object in the notebook is as simple as putting it on the end of a cell. Note, however, that the ``panel.extension`` first has to be loaded to initialize the required JavaScript in the notebook context.  Also, if you are working in JupyterLab, the pyviz labextension has to be installed with:

    jupyter labextension install @pyviz/jupyterlab_pyviz

### Optional dependencies

Also remember that in order to use certain components such as Vega, LaTeX, and Plotly plots in a notebook, the models must be loaded using the extension. If you forget to load the extension, you should get a warning reminding you to do it. To load certain JS components, simply list them as part of the call to ``pn.extension``:

    pn.extension('vega', 'katex')

Here we've ensured that the Vega and LaTeX JS dependencies will be loaded.

### Initializing JS and CSS 

Additionally, any external ``css_files``, ``js_files`` and ``raw_css`` needed should be declared in the extension. The ``js_files`` should be declared as a dictionary mapping from the exported JS module name to the URL containing the JS components, while the ``css_files`` can be defined as a list:

    pn.extension(js_files={'deck': https://unpkg.com/deck.gl@~5.2.0/deckgl.min.js},
                 css_files=['https://api.tiles.mapbox.com/mapbox-gl-js/v0.44.1/mapbox-gl.css'])

The ``raw_css`` argument allows defining a list of strings containing CSS to publish as part of the notebook and app.

Providing keyword arguments via the ``extension`` is the same as setting them on ``pn.config``, which is the preferred approach outside the notebook. ``js_files`` and ``css_files`` may be set to your chosen values as follows:

    pn.config.js_files  = {'deck': 'https://unpkg.com/deck.gl@~5.2.0/deckgl.min.js'}
    pn.config.css_files = ['https://api.tiles.mapbox.com/mapbox-gl-js/v0.44.1/mapbox-gl.css']

## Display in the notebook

#### The repr
    
Once the extension is loaded, Panel objects will display themselves if placed at the end of cell in the notebook:

In [ ]:
pane = pn.panel('<marquee>Here is some custom HTML</marquee>')

pane

To instead see a textual representation of the component, you can use the ``pprint`` method on any Panel object:

In [ ]:
pane.pprint()

#### The ``display`` function

To avoid having to put a Panel on the last line of a notebook cell, e.g. to display it from inside a function call, you can use the IPython built-in ``display`` function:

In [ ]:
def display_marquee(text):
    display(pn.panel('<marquee>{text}</marquee>'.format(text=text)))
    
display_marquee('This Panel was displayed from within a function')

#### Inline apps

Lastly it is also possible to display a Panel object as a Bokeh server app inside the notebook.  To do so call the ``.app`` method on the Panel object and provide the URL of your notebook server:

In [ ]:
pane.app('localhost:8888')

The app will now run on a Bokeh server instance separate from the Jupyter notebook kernel, allowing you to quickly test that all the functionality of your app works both in a notebook and in a server context.

## Display in the Python REPL

Working from the command line will not automatically display rich representations inline as in a notebook, but you can still interact with your Panel components if you start a Bokeh server instance and open a separate browser window using the ``show`` method. The method has the following arguments:

    port: int (optional)
       Allows specifying a specific port (default=0 chooses an arbitrary open port)
    websocket_origin: str or list(str) (optional)
       A list of hosts that can connect to the websocket.
       This is typically required when embedding a server app in
       an external-facing web site.
       If None, "localhost" is used.
    threaded: boolean (optional, default=False)
       Whether to launch the Server on a separate thread, allowing
       interactive use.
       
To work with an app completely interactively you can set ``threaded=True`,` which will launch the server on a separate thread and let you interactively play with the app.

<img src='https://assets.holoviews.org/panel/gifs/commandline_show.gif'></img>

The ``show`` call will return either a Bokeh server instance (if ``threaded=False``) or a ``StoppableThread`` instance (if ``threaded=True``) which both provide a ``stop`` method to stop the server instance.

## Launching a server on the commandline

Once the app is ready for deployment it can be served using the Bokeh server.  For a detailed breakdown of the design and functionality of Bokeh server, see the [Bokeh documentation](https://bokeh.pydata.org/en/latest/docs/user_guide/server.html). The most important thing to know is that Panel (and Bokeh) provide a CLI command to serve a Python script, app directory, or Jupyter notebook containing a Bokeh or Panel app. To launch a server using the CLI, simply run:

    panel serve app.ipynb
    
The ``panel serve`` command has the following options:

    positional arguments:
      DIRECTORY-OR-SCRIPT   The app directories or scripts or notebooks to serve 
                            (serve empty document if not specified)

    optional arguments:
      -h, --help            show this help message and exit
      --port PORT           Port to listen on
      --address ADDRESS     Address to listen on
      --log-level LOG-LEVEL
                            One of: trace, debug, info, warning, error or critical
      --log-format LOG-FORMAT
                            A standard Python logging format string (default:
                            '%(asctime)s %(message)s')
      --log-file LOG-FILE   A filename to write logs to, or None to write to the
                            standard stream (default: None)
      --args ...            Any command line arguments remaining are passed on to
                            the application handler
      --show                Open server app(s) in a browser
      --allow-websocket-origin HOST[:PORT]
                            Public hostnames which may connect to the Bokeh
                            websocket
      --prefix PREFIX       URL prefix for Bokeh server URLs
      --keep-alive MILLISECONDS
                            How often to send a keep-alive ping to clients, 0 to
                            disable.
      --check-unused-sessions MILLISECONDS
                            How often to check for unused sessions
      --unused-session-lifetime MILLISECONDS
                            How long unused sessions last
      --stats-log-frequency MILLISECONDS
                            How often to log stats
      --mem-log-frequency MILLISECONDS
                            How often to log memory usage information
      --use-xheaders        Prefer X-headers for IP/protocol information
      --session-ids MODE    One of: unsigned, signed, or external-signed
      --index INDEX         Path to a template to use for the site index
      --disable-index       Do not use the default index on the root path
      --disable-index-redirect
                            Do not redirect to running app from root path
      --num-procs N         Number of worker processes for an app. Using 0 will
                            autodetect number of cores (defaults to 1)
      --websocket-max-message-size BYTES
                            Set the Tornado websocket_max_message_size value
                            (defaults to 20MB) NOTE: This setting has effect ONLY
                            for Tornado>=4.5
      --dev [FILES-TO-WATCH [FILES-TO-WATCH ...]]
                            Enable live reloading during app development.By
                            default it watches all *.py *.html *.css *.yaml
                            filesin the app directory tree. Additional files can
                            be passedas arguments. NOTE: This setting only works
                            with a single app.It also restricts the number of
                            processes to 1.

To turn a notebook into a deployable app simply append ``.servable()`` to one or more Panel objects, which will add the app to Bokeh's ``curdoc``, ensuring it can be discovered by Bokeh server on deployment. In this way it is trivial to build dashboards that can be used interactively in a notebook and then seamlessly deployed on Bokeh server.

When called on a notebook, `panel serve` first converts it to a python script using [`nbconvert.PythonExporter()`](https://nbconvert.readthedocs.io/en/stable/api/exporters.html), albeit with [IPython magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html) stripped out. This means that non-code cells, such as raw cells, are entirely handled by `nbconvert` and [may modify the served app](https://nbsphinx.readthedocs.io/en/latest/raw-cells.html).

### Accessing session state

Whenever a Panel app is being served the ``panel.state`` object exposes some of the internal Bokeh server components to a user.

#### Document

The current Bokeh ``Document`` can be accessed using ``panel.state.curdoc``.

#### Request arguments

When a browser makes a request to a Bokeh server a session is created for the Panel application. The request arguments are made available to be accessed on ``pn.state.session_args``. For example if your application is hosted at ``localhost:8001/app``, appending ``?phase=0.5`` to the URL will allow you to access the phase variable using the following code:

```python
try:
    phase = int(pn.state.session_args.get('phase')[0])
except:
    phase = 1
```

This mechanism may be used to modify the behavior of an app dependending on parameters provided in the URL. 

### Accessing the Bokeh model

Since Panel is built on top of Bokeh, all Panel objects can easily be converted to a Bokeh model. The ``get_root`` method returns a model representing the contents of a Panel:

In [ ]:
pn.Column('# Some markdown').get_root()

By default this model will be associated with Bokeh's ``curdoc()``, so if you want to associate the model with some other ``Document`` ensure you supply it explictly as the first argument.

## Embedding

Panel generally relies on either the Jupyter kernel or a Bokeh Server to be running in the background to provide interactive behavior. However for simple apps with a limited amount of state it is also possible to `embed` all the widget state, allowing the app to be used entirely from within Javascript. To demonstrate this we will create a simple app which simply takes a slider value, multiplies it by 5 and then display the result.

In [ ]:
slider = pn.widgets.IntSlider(start=0, end=10)

@pn.depends(slider.param.value)
def callback(value):
    return '%d * 5 = %d' % (value, value*5)

row = pn.Row(slider, callback)

If we displayed this the normal way it would call back into Python every time the value changed. However, the `.embed()` method will record the state of the app for the different widget configurations.

In [ ]:
row.embed()

If you try the widget above you will note that it only has 3 different states, 0, 5 and 10. This is because by default embed will try to limit the number of options of non-discrete or semi-discrete widgets to at most three values. This can be controlled using the `max_opts` argument to the embed method. The full set of options for the embed method include:

- **max_states**: The maximum number of states to embed

- **max_opts**: The maximum number of states for a single widget

- **json** (default=True): Whether to export the data to json files

- **save_path** (default='./'): The path to save json files to

- **load_path** (default=None):  The path or URL the json files will be loaded from (same as ``save_path`` if not specified)

As you might imagine if there are multiple widgets there can quickly be a combinatorial explosion of states so by default the output is limited to about 1000 states. For larger apps the states can also be exported to json files, e.g. if you want to serve the app on a website specify the ``save_path`` to declare where it will be stored and the ``load_path`` to declare where the JS code running on the website will look for the files.

## Saving 

In case you don't need an actual server or simply want to export a static snapshot of a panel app, you can use the ``save`` method, which allows exporting the app to a standalone HTML or PNG file.

By default, the HTML file generated will depend on loading JavaScript code for BokehJS from the online ``CDN`` repository, to reduce the file size. If you need to work in an airgapped or no-network environment, you can declare that ``INLINE`` resources should be used instead of ``CDN``:

```python
from bokeh.resources import INLINE
panel.save('test.html', resources=INLINE)
```

Additionally the save method also allows enabling the `embed` option, which, as explained above, will embed the apps state in the app or save the state to json files which you can ship alongside the exported HTML.

Finally, if a 'png' file extension is specified, the exported plot will be rendered as a PNG, which currently requires Selenium and PhantomJS to be installed:

```python
pane.save('test.png')

```